# Test 2. qutip ```norm()``` function

The norm of a state $\rho$ is defined as $\| \rho \| \equiv \sqrt{\rho^{\dagger} \rho}$

In [7]:
import qutip as qu
import numpy as np
import matplotlib.pyplot as plt
# from scipy.special import comb, factorial

# from QI import *

In [2]:
state = coherent_dm(5,1)

In [3]:
state

Quantum object: dims = [[5], [5]], shape = [5, 5], type = oper, isherm = True
Qobj data =
[[ 0.36791117  0.36774407  0.26105441  0.14620658  0.08826704]
 [ 0.36774407  0.36757705  0.26093584  0.14614018  0.08822695]
 [ 0.26105441  0.26093584  0.18523331  0.10374209  0.06263061]
 [ 0.14620658  0.14614018  0.10374209  0.05810197  0.035077  ]
 [ 0.08826704  0.08822695  0.06263061  0.035077    0.0211765 ]]

In [4]:
state.norm()

1.0

In [5]:
state_test = basis(2,0) + basis(2,1)

In [6]:
state_test

Quantum object: dims = [[2], [1]], shape = [2, 1], type = ket
Qobj data =
[[ 1.]
 [ 1.]]

In [7]:
state_test.norm()

1.4142135623730951

# Test 3. Quitp `eigenstates()` function

In [1]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
# from scipy.special import comb, factorial

from QI import *

In [2]:
Nth = 0.1     # Average thermal photon numbers
N = 10        # Truncated photon numbers, i.e., 0 ~ N-1
eta = 0.01    # Transmissivity

Ns = 0.01     # Average photon number of the initial two-mode squeezed states
lmd = np.sqrt(Ns/(1 + Ns))     # 
s = np.arcsinh(np.sqrt(Ns))    # Squeezed parameter

In [3]:
rho_0 = RHO_0(TMSS, N, lmd, Nth)
rho_1 = RHO_1(TMSS, N, lmd, Nth, eta)

In [11]:
rho_0.data

<100x100 sparse matrix of type '<type 'numpy.complex128'>'
	with 40 stored elements in Compressed Sparse Row format>

In [12]:
rho_1.data

<100x100 sparse matrix of type '<type 'numpy.complex128'>'
	with 226 stored elements in Compressed Sparse Row format>

In [4]:
gamma = 0.5*rho_1 - 0.5*rho_0

In [6]:
eigval, eigvect = gamma.eigenstates(sparse=True, sort='high')

In [7]:
sum(abs(eigval))

0.0094413487267620104

In [8]:
gamma.norm()

0.0094413487267620087

In [9]:
(1 - gamma.norm()) / 2.0

0.49527932563661897

In [10]:
Helstrom(0.5, rho_0, 0.5, rho_1)

0.49527932563661897

In [13]:
0.5 * QCB(rho_0, rho_1, approx=True)

0.49995710002428295

# Test 4. Two Mode Mixing

In [8]:
def tm_mix(s, n_max):
    """
    Two-mode mixing operator (Beam splitter)

    Parameters
    ----------
    s: float (TODO: a complex number??)
        The mixing parameter
    n_max: positive interger
        Photon numbers can be in [0, n_max - 1]

    Return
    ------
        a new qubit.Qobj()
    """
    a = qu.destroy(n_max)
    tmm = s * qu.tensor(a.dag(), a) - np.conj(s) * qu.tensor(a, a.dag())
    return tmm.expm()

In [9]:
def tm_sqz(s, n_max):
    """
    Two mode mixing operator in the form of matrix with Fock basis

    Parameters
    ----------
    s: float (TODO: a complex number??)
        The squeezed parameter
    n_max: positive integer
        Photon numbers can be in [0, n_max - 1]

    Return
    ------
        a new qubit.Qobj()
    """
    a = qu.destroy(n_max)
    tms = - np.conj(s) * qu.tensor(a, a) + s * qu.tensor(a.dag(), a.dag())
    return tms.expm()

### Two mode squeezing

In [65]:
n_max = 20
lmd = np.sqrt(0.01/1.01)

# Test two-mode squeezing
s = np.arctanh(lmd)
input2 = qu.ket2dm(qu.tensor(qu.basis(n_max, 0), qu.basis(n_max, 0)))
tm_sqz_op = tm_sqz(s, n_max)
output2 = tm_sqz_op * input2 * tm_sqz_op.dag()
print("\nEntropy of entanglement: {}".format(qu.entropy_vn(output2.ptrace(1))))


Entropy of entanglement: 0.05610153599544214


In [66]:
output2.norm()

1.0000000000102738

In [67]:
output2.ptrace(0).norm()

0.99999999999905775

In [68]:
output2.ptrace(1).norm()

0.99999999999905775

In [69]:
qu.expect(qu.num(n_max), output2.ptrace(0)) 

0.009999999994338307

In [70]:
qu.expect(qu.num(n_max), output2.ptrace(1))

0.009999999994338307

### Two mode mixing

In [71]:
reflectance = 0.01
nth = 1.0
thermal_1 = qu.thermal_dm(n_max, nth / (1 - reflectance)).unit()
rho_ab = output2.unit()
rho_abc = qu.tensor(rho_ab, thermal_1)

In [72]:
xi = np.arccos(np.sqrt(reflectance))
op = qu.tensor(qu.qeye(n_max), tm_mix(xi, n_max))
rho_1 = op * rho_abc * op.dag()

In [73]:
n0 = qu.expect(qu.num(n_max), rho_1.ptrace(0))
n1 = qu.expect(qu.num(n_max), rho_1.ptrace(1))
n2 = qu.expect(qu.num(n_max), rho_1.ptrace(2))
print(n0, n1, n2)

0.009999999872344211 1.0000791136732745 0.02000081109909703


In [74]:
xi = np.arcsin(np.sqrt(reflectance))
op = qu.tensor(qu.qeye(n_max), tm_mix(xi, n_max))
rho_1 = op * rho_abc * op.dag()

In [75]:
n0 = qu.expect(qu.num(n_max), rho_1.ptrace(0))
n1 = qu.expect(qu.num(n_max), rho_1.ptrace(1))
n2 = qu.expect(qu.num(n_max), rho_1.ptrace(2))
print(n0, n1, n2)

0.009999999871345118 0.020000801043208487 1.0000791237379052
